### Factors that affect commission rate

Commission rate: date, customers name, car make, car model, car year, sale price, number of car sold of the salesperson

In [ ]:
import pandas as pd
df = pd.read_csv('./datasets/car_sales_data.csv')

In [ ]:
# Calculate number of car sold of the sales person

In [ ]:
salesperson_counts = df.groupby('Salesperson').agg({'Date': 'count'}).reset_index()
salesperson_counts.rename(columns={'Date': 'Ncar_of_salesperson'}, inplace=True)

In [ ]:
df = pd.merge(df, salesperson_counts, left_on='Salesperson', right_on = 'Salesperson', how='left')

In [ ]:
# Commission rate: date, customers name, car make, car model, car year, sale price, number of car sold of the salesperson

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

label_encoder = LabelEncoder()
onehot = OneHotEncoder()

In [ ]:
columns_to_encode = ['Customer Name','Car Year', 'Date']

for column in columns_to_encode:
    df[column] = label_encoder.fit_transform(df[column])

In [ ]:
columns_to_onehot_encode = ['Car Make', 'Car Model']

for column in columns_to_onehot_encode:
    new_col = pd.DataFrame(onehot.fit_transform(df[[column]]).toarray(), columns = onehot.get_feature_names_out())
    df.drop(column, axis = 1, inplace = True)
    df = df.join(new_col)

In [ ]:
dep_var = 'Commission Rate'

In [ ]:
ind_var = list(df.columns)

In [ ]:
ind_var.remove(dep_var)

In [ ]:
ind_var.remove('Commission Earned')
ind_var.remove('Salesperson')

In [ ]:
ind_var

['Date',
 'Customer Name',
 'Car Year',
 'Sale Price',
 'Ncar_of_salesperson',
 'Car Make_Chevrolet',
 'Car Make_Ford',
 'Car Make_Honda',
 'Car Make_Nissan',
 'Car Make_Toyota',
 'Car Model_Altima',
 'Car Model_Civic',
 'Car Model_Corolla',
 'Car Model_F-150',
 'Car Model_Silverado']

In [ ]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(df, train_size = 0.8, random_state=10, shuffle=True)
X_train, X_test = train_data[ind_var], test_data[ind_var]
y_train, y_test = train_data[dep_var], test_data[dep_var]

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.neural_network import MLPRegressor

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid = {'alpha':[.0001, 0.001, 0.01, 0.1, 1]}

In [ ]:
grid = GridSearchCV(Ridge(), param_grid, scoring='r2')
grid.fit(X_train_scaled, y_train)

GridSearchCV(estimator=Ridge(),
             param_grid={'alpha': [0.0001, 0.001, 0.01, 0.1, 1]}, scoring='r2')

In [ ]:
print(grid.best_params_)

{'alpha': 1}


In [ ]:
from sklearn.metrics import make_scorer
import numpy as np

# Define custom MAPE scoring function
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Make it a scorer using make_scorer
mape_scorer = make_scorer(mape, greater_is_better=False)

In [ ]:
grid = GridSearchCV(Lasso(), param_grid, scoring=mape_scorer)
grid.fit(X_train_scaled, y_train)

GridSearchCV(estimator=Lasso(),
             param_grid={'alpha': [0.0001, 0.001, 0.01, 0.1, 1]},
             scoring=make_scorer(mape, greater_is_better=False))

In [ ]:
print(grid.best_params_)

{'alpha': 0.0001}


In [ ]:
models = {
    'Linear Regression': LinearRegression(),
    'Polynominal Features': LinearRegression(),
    'Ridge Regression': Ridge(alpha=1),
    'Lasso Regression': Lasso(alpha=0.0001),
    'neural_network': MLPRegressor()
}

In [ ]:
def model_test(model, X_train, X_test, y_train, y_test):

    model.fit(X_train, y_train)
    y_test_predicted = model.predict(X_test)
    y_train_predicted = model.predict(X_train)

    score = []
    score.append(mean_absolute_percentage_error(y_test, y_test_predicted))
    score.append(mean_absolute_percentage_error(y_train, y_train_predicted))
    score.append(mean_squared_error(y_test, y_test_predicted)**0.5)
    score.append(mean_squared_error(y_train, y_train_predicted)**0.5)
    score.append(r2_score(y_test, y_test_predicted))
    score.append(r2_score(y_train, y_train_predicted))

    return score

In [ ]:
df1 = pd.DataFrame(index = ['mape_test', 'mape_train', 'rmse_test', 'rmse_train', 'R_squared_test', 'R_squared_train'])

In [ ]:
for name, model in models.items():
    if name == 'Polynominal Features':
        poly_features = PolynomialFeatures(degree=2)
        X_train_poly = poly_features.fit_transform(X_train_scaled)
        X_test_poly = poly_features.fit_transform(X_test_scaled)
        df1[name] = model_test(model, X_train_poly, X_test_poly, y_train, y_test)
    else:
        df1[name] = model_test(model, X_train_scaled, X_test_scaled, y_train, y_test)

In [ ]:
df1

,Linear Regression,Polynominal Features,Ridge Regression,Lasso Regression,neural_network
mape_test,0.288140,0.288134,0.288140,0.288141,0.284696
mape_train,0.287582,0.287556,0.287582,0.287583,0.284143
rmse_test,0.028871,0.028871,0.028871,0.028871,0.028896
rmse_train,0.028859,0.028858,0.028859,0.028859,0.028887
R_squared_test,0.000003,-0.000015,0.000005,-0.000001,-0.001791
R_squared_train,0.000005,0.000083,0.000005,0.000000,-0.001881
